# Предобработка данных.

# Проект "Исследование надежности заемщиков банка"

1. [Описание проекта](#intro)    
2. [Загрузка и изучение данных](#data)        
    2.1. [Вывод](#conclusion1)     
3. [Предобработка данных](#preprocessing)    
    3.1. [Обработка пропусков](#gaps)    
    3.2. [Обработка аномалий](#anomalies)    
    3.3. [Обработка типов данных](#types)    
    3.4. [Удаление дубликатов](#duplicates)    
    3.5. [Исключение дублирующейся информации](#id)    
    3.6. [Категоризация клиентов по уровню дохода](#income)    
    3.7. [Категоризация клиентов по цели кредита](#purpose)    
    3.8. [Вывод](#conclusion2)    
4. [Исследовательский анализ данных](#eda)        
    4.1. [Вывод](#conclusion3)    
5. [Выводы и рекомендации](#conclusion_all)    

<a id="intro"></a>

## Описание проекта 

Заказчик — кредитный отдел банка. Необходимо изучить, как влияют различные параметры клиентов на вероятность возврата кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель:**
Изучить влияение семейного положения, количества детей, уровня дохода и цели кредита клиента на факт погашения кредита в срок

**Задачи:**
1. Изучить данные
2. Провести тщательную предобработку данных, включающую в себя:
    - Анализ пропусков
    - Поиск артефактов (аномалий)
    - Проверка типов записи данных
    - Поиск дубликатов
3. Заменить строковые переменные идентификаторами
4. Провести категоризацию клиентов по уровню дохода
5. Провести категоризацию клиентов по цели открытия кредита


**Описание данных**

Входные данные от банка — статистика о платёжеспособности клиентов
 
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

**Инструментарий:**

- *библиотеки:* pandas
- *работа с данными:* read_csv, info, try-except, isna, sum, fillna, duplicated, drop_duplicated, reset_index, sort_values, groupby, len, transform, loc, value_counts, abs, astype, str.lower, dropna, apply, pivot_table, merge
- *циклы for, определение функций, конструкция if-else*

<a id="data"></a>

## Обзор и изучение данных

Импортируем библиотеки и выключим отображение предупреждений

In [26]:
import pandas as pd
import warnings

In [27]:
# Выключение отображения предупреждений
warnings.filterwarnings('ignore')

Загрузим базу данных из файла

In [2]:
# чтение файла с данными и сохранение в data с сервера или локального компьютера
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('E:/IT/Yandex/Projects/datasets/bank_credit.csv')

In [21]:
# Определим функцию для знакомства с базой данных и проверки её на явные дубликаты
def database_check(db):
    print('Случайные 5 строк базы данных')
    display(db.sample(5, random_state=0))
    print('---')
    print('Общая информация о базе данных')
    db.info()
    print('---')
    print('Пропущенных значений в строках:')
    print(db.isna().sum())
    print('---')
    print('Явных дубликатов в базе данных:', db.duplicated().sum())

Изучим базу данных

In [24]:
database_check(data)

Случайные 5 строк базы данных


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
1514,0,269,26,1,0,M,сотрудник,1,126414,операции с жильем,C,операции с недвижимостью
1191,0,1630,61,1,0,F,пенсионер,0,118514,операции с недвижимостью,C,операции с недвижимостью
9806,1,1630,38,0,0,M,сотрудник,0,142594,операции со своей недвижимостью,C,операции с недвижимостью
20662,0,550,61,1,0,F,компаньон,0,64115,ремонт жилью,C,операции с недвижимостью
1748,0,229,45,0,0,F,сотрудник,0,151347,получение образования,C,получение образования


---
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21454 non-null  int64 
 1   days_employed          21454 non-null  int32 
 2   dob_years              21454 non-null  int64 
 3   education_id           21454 non-null  int64 
 4   family_status_id       21454 non-null  int64 
 5   gender                 21454 non-null  object
 6   income_type            21454 non-null  object
 7   debt                   21454 non-null  int64 
 8   total_income           21454 non-null  int32 
 9   purpose                21454 non-null  object
 10  total_income_category  21454 non-null  object
 11  purpose_category       21454 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB
---
Пропущенных значений в строках:
children                 0
days_employed      

<a id="conclusion1"></a>

### Вывод

Названия столбцов записаны верно. В столбцах days_employed и total_income пропущено по 2174 значений. Также указанные поля имеют тип данных float64, что значительно повышает размер данных, при этом целесообразность записи этих значений действительными числами, возможно, не оправдана и можно их записать через целочисленные значения. 

В таблице присутствует 54 явных дубликата, которые необходимо будет удалить.    

<a id="preprocessing"></a>

## Предобработка данных

<a id="gaps"></a>

### Обработка пропусков

Изучим, в каких столбцах данные пропущены и как лучше поступить, чтобы уменьшить их долю

In [5]:
print('Всего значений в таблице:', len(data))
print('---')
print('Пропущенных значений в таблице:')
display(data.isnull().sum())
percentage_na = int(data['days_employed'].isnull().sum() / len(data) * 100)
print('---')
print(f'Доля пропущенных значений: {percentage_na}%')

Всего значений в таблице: 21525
---
Пропущенных значений в таблице:


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

---
Доля пропущенных значений: 10%


В столбцах days_employed и total_income пропущено 2174 строки, что составляет 10% от общего количества записей.
Видимых причин связи пропущенных данных с данными в других столбцах не обнаружено, поэтому наиболее вероятная причина пропусков - человеческий фактор.
Проверим, сколько записей соответствует каждому виду занятости

In [6]:
print('Количество записей, сгруппированных по типу дохода и показатель их среднего количества дней стажа и дохода')
display(data.groupby(['income_type']).agg({'income_type': ['count'],'days_employed': ['median'], 'total_income': ['median']}))

Количество записей, сгруппированных по типу дохода и показатель их среднего количества дней стажа и дохода


,income_type,days_employed,total_income
,count,median,median
income_type,,,
безработный,2,366413.652744,131339.751676
в декрете,1,-3296.759962,53829.130729
госслужащий,1459,-2689.368353,150447.935283
компаньон,5085,-1547.382223,172357.950966
пенсионер,3856,365213.306266,118514.486412
предприниматель,2,-520.848083,499163.144947
сотрудник,11119,-1574.202821,142594.396847
студент,1,-578.751554,98201.625314


Для заполнения пропусков в столбце total_income лучше использовать медианные значения, потому что так данные не подвержены аномальным очень высоким или низких значениям, значит, лучше предоставляют информацию о средних значениях.

In [7]:
#Заменим пропущенные значения медианными значениями в каждой из групп по типу заработка: 
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform("median"))

#Проверим, что все пропущенные значения заменены на средние
print('Пропущенных значений в графе среднего дохода:', data['total_income'].isnull().sum())

Пропущенных значений в графе среднего дохода: 0


<a id="anomalies"></a>

### Обработка аномалий

Группировка данных по медианному значению дней стажа у каждой группы по типу занятости выявила, что положительные значения наблюдаются у безработных и пенсионеров. Посколькду эти группы не принадлежат к работающим, то количество дней стажа для них не играет роли, и его можно приравнять к среднему значению.
Природу появления отрицательных значений для остальных групп понять невозможно, но если перевести дни в года, то получаются в целом логичные значения. За неимением другой информации, можем предположить, что произошла системная ошибка и все положительные значения поменялись на отрицательные, следовательно, чтобы восстановить логику в данных, домножим все отрицательные значения на -1, чтобы получить положительные значения.
Все пропущенные значения заменим медианным средним по выборке.

In [8]:
#Домножаем все значения в столбце средних зарплат на -1
data['days_employed'] = data['days_employed']*(-1)
#Расчитываем среднее значение трудового стажа
median_days_employed = data.loc[data['days_employed'] > 0, 'days_employed'].median()
#Заменяем все значения для пенсионеров и безработных средним значением
data.loc[data['days_employed'] < 0, 'days_employed'] = median_days_employed
#Заменяем все пропущенные значения средним значением
data['days_employed'] = data['days_employed'].fillna(value=median_days_employed)

print('Пропущенных значений в графе трудового стажа:', data['days_employed'].isnull().sum())

Пропущенных значений в графе трудового стажа: 0


In [9]:
#Проверим графу дети
print('Количество детей у заёмщиков')
display(data['children'].value_counts())

Количество детей у заёмщиков


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения -1 и 20 явные аномалии. Можем предположить, что данные записаны некоректно, и -1 означает 1, а 20 - 2. Заменим значения в базе данных

In [10]:
data.loc[data['children'] == 20, 'children'] = 2
data['children'] = abs(data['children'])

<a id="types"></a>

### Обработка типов данных

Данные в столбцах days_employed и total_income представлены в формате float64, что расходует ресурсы памяти, при этом не привнося значительной информации. Для экономиии ресурсов и удобности прочтения, можно заменить тип данных в этих столбцах на целочисленный

In [11]:
#Замена дней стажа на целочисленные значения
data['days_employed'] = data['days_employed'].astype(int)
#Замена дохода на целочисленные значения
data['total_income'] = data['total_income'].astype(int)

<a id="duplicates"></a>

### Удаление дубликатов

Чтобы найти дубликаты, в начале нужно исследовать данные. Качественные переменные, в которых наиболее вероятно могут быть встречены ошибки ввода, присутствуют в следующих столбцах: education, family_status, income_type, purpose. Проверим уникальные значения, которые встречаются в этих столбцах

In [12]:
print('Значения в столбце уровень образования клиента:')
display(data['education'].value_counts())
print('-----')
print('Значения в столбце семейное положение:')
display(data['family_status'].value_counts())
print('-----')
print('Значения в столбце тип занятости:')
display(data['income_type'].value_counts())
print('-----')
print('Значения в столбце цель получения кредита:')
display(data['purpose'].value_counts())

Значения в столбце уровень образования клиента:


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

-----
Значения в столбце семейное положение:


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

-----
Значения в столбце тип занятости:


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

-----
Значения в столбце цель получения кредита:


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце уровень образования множество дубликатов, связанных с регистром. В столбцах семейное положение и тип занятости дубликатов нет, в столбце цель получения кредита имеются разные формулировки одних и тех же целей.
Удалим явные дубликаты и дубликаты, связанные с регистром записи графы образование

In [13]:
#Переведем все значения столбца образование в нижний регистр
data['education'] = data['education'].str.lower()
#Заодно унифицируем регистр в столбце семейного положения
data['family_status'] = data['family_status'].str.lower()
#Удалим дубликаты
print('Дубликатов в таблице было:', data.duplicated().sum())
data = data.drop_duplicates().dropna().reset_index(drop=True)
print('Дубликатов в таблице стало:', data.duplicated().sum())

Дубликатов в таблице было: 71
Дубликатов в таблице стало: 0


Для поиска и удаления дубликатов был применен метод drop_duplicates в совокупности с методом dropna, что позволило вычислить явные дубликаты, очистить их значения и удалить очищенные ячейки. Чтобы индексация после удаления была без пропусков, дополнительно был применен метод reset_index. Выполнение операции проверено методом duplicated.
Появление дубликатов можно объяснить ошибкой ввода (человеческий фактор), когда один и тот же человек ввел свои данные два раза при регистрации. И технической ошибкой, например, при переносе из одной базы данных в другую, где-то по умолчанию мог стоять верхний регистр в записи атрибутов, а где-то нижний. Но общее количество дубликатов (71 запись из 21252) говорит о высоком качестве предоставленной базы данных и что эти дубликаты не оказывают существенного влияния на способ решения поставленной задачи.

<a id="id"></a>

### Исключение дублирующейся информации

Сформируем дополнительные датафреймы "словарей" для полей education и family_status и удалим описательные данные в исходной базе, оставив только категориальные.

In [14]:
#Создание 'словаря' для уровня образования
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
#Создание 'словаря' для семейного положения
family_dict = data[['family_status', 'family_status_id']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
#Удаление столбцов education и family_status из исходной таблицы
data = data.drop('education', 1)
data = data.drop('family_status', 1)

<a id="income"></a>

### Категоризация клиентов по уровню дохода

Проведем категоризацию клиентов банка по уровню дохода, выделив 5 групп

In [15]:
#Функция для категоризации дохода
def income_category_func(income):
    if (income > 30000 and income <= 50000):
        return 'D'
    elif (income > 50000 and income <= 200000):
        return 'C'
    elif (income > 200000 and income <= 1000000):
        return 'B'
    elif (income > 1000000):
        return 'A'
    else:
        return 'E'
    
#Применение функции в новом столбце total_income_category
data['total_income_category'] = data['total_income'].apply(income_category_func)

<a id="purpose"></a>

### Категоризация клиентов по цели кредита

Проведем категоризацию клиентов банка по уровню дохода, выделив 5 групп:
- Свадьба
- Недвижимость
- Автомобиль
- Образование
- Другое

In [16]:
#Функция для категоризации целей получения кредита
def purpose_category_func(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'образов' in purpose:
        return 'получение образования'
    else:
        return 'другое'

#Применение функции к столбцу целей и запись в новый столбец
data['purpose_category'] = data['purpose'].apply(purpose_category_func)

<a id="conclusion1"></a>

### Вывод

В столбцах days_employed и total_income содержалось по 2174 пропущенных записей (10% от общего числа), которые были заменены на медианные значения по типу дохода. В столбце children содержались аномальные значения (-1, 20), которые были заменены на 1 и 2, сооветственно. Тип данных в столбцах days_employed и total_income был заменен на целочисленный без потери количества информации. В столбцах education и family_status различные записи данных были унифицированы, что позволило выявить ещё 71 неявный дубликат, после чего дубликаты были удалены из базы данных.

Столбцы education и education_id, а так же family_status и family_status_id содержат дублирующуюся информацию. Для более компактного отображения, для этих столбцов были созданы "словари" соответствия и строковые столбцы удалены из рабочей базы данных.

Для удобства анализа, данные о доходе клиентов и цели взятия кредита были категоризированны в 5 групп. 

<a id="eda"></a>

## Исследовательский анализ данных

Сравним показатель возврата кредита в срок (debt) с интересующими нас параметрами: 
- количество детей, 
- семейное положение, 
- уровень дохода,
- цель кредита.

Проверим связь показателя невозврата долгов с количеством детей

In [17]:
# Сгруппируем данные по количеству детей
data_pivot_children = data.pivot_table(index=['children'], columns= 'debt', values='days_employed', aggfunc='count')
# Рассчитаем процент заёмщиков в каждой группе, кто не вернул долг
data_pivot_children['children_ratio_debt'] = data_pivot_children[1] / (data_pivot_children[0] + data_pivot_children[1]) * 100
print('Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по количеству детей')
display(data_pivot_children['children_ratio_debt'].sort_values(ascending=False))

Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по количеству детей


children
4    9.756098
2    9.492481
1    9.165808
3    8.181818
0    7.543822
5         NaN
Name: children_ratio_debt, dtype: float64

Чтобы понять, какие данные репрезентативны для сравнения, можно сравнить количество записей, соответствующих каждому параметру. В связи с тем, что данные с количеством детей от 0 до 2 на порядок больше, чем остальных значений, то имеет смысл делать выводы в первую очередь опираясь на них.

Если сравнивать 3 репрезентативные группы (без детей, 1 ребенок или 2 ребенка), то вырисовывается четкая связь параметра с долгами по кредиту: чем больше детей, тем больше вероятность появления долга. При этом риск просрочки кредита у людей с детьми примерно одинаковый и заметно выше (на 2%), чем у тех, кто без детей.

Остальные ключевые параметры представлены качественными категориями, а не количественными показателями, поэтому можно предположить, что выборки в них репрезентативны для всех категорий.

Проверим связь показателя невозврата долгов с семейным положением

In [18]:
#Группировка данных по семейному положению
data_pivot_family = data.pivot_table(index=['family_status_id'], columns='debt', values='days_employed', aggfunc='count')
#Расчет доли невозврата долгов
data_pivot_family['family_ratio_debt'] = data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1]) * 100
data_pivot_family = data_pivot_family.merge(family_dict, on='family_status_id', how='left')
data_pivot_family = data_pivot_family[['family_status', 'family_ratio_debt']]

print('Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по семейному положению')
display(data_pivot_family.sort_values(by='family_ratio_debt', ascending=False))

Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по семейному положению


,family_status,family_ratio_debt
4,не женат / не замужем,9.750890
1,гражданский брак,9.347145
0,женат / замужем,7.545182
3,в разводе,7.112971
2,вдовец / вдова,6.569343


Чёткой связи по параметру семейного положения не наблюдается. Можно условно поделить на две категории: 1) люди, не имевшие опыт юридически оформленного брака, 2) люди, имеющие или имевшие в прошлом такой опыт. В первой категории средний риск просрочки кредита на 2% выше, чем во второй.

Проверим связь показателя невозврата долгов с уровнем дохода

In [19]:
#Группировка данных по уровню дохода
data_pivot_income = data.pivot_table(index=['total_income_category'], columns= 'debt', values='days_employed', aggfunc='count')
#Расчет доли невозврата по уровню дохода
data_pivot_income['income_ratio_debt'] = data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1]) * 100

print('Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по уровню дохода')
display(data_pivot_income['income_ratio_debt'].sort_values(ascending=False))

Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по уровню дохода


total_income_category
E    9.090909
C    8.492039
A    8.000000
B    7.060690
D    6.000000
Name: income_ratio_debt, dtype: float64

Как ни странно, по уровню дохода четкой связи тоже не наблюдается. Закономерно, в зоне наибольшего риска наименее обеспеченная группа граждан (категория E). При этом наиболее благонадежные граждане оказываются в группе D, то есть имеющие доход немногим выше минимального. Если судить по наиболее распространенным уровням дохода (категории В и С), то здесь наблюдается логика: в категории с более высоким уровнем дохода вероятность просрочки на 1,5% ниже, чем в категории с более низким уровнем.

Проверим связь показателя невозврата долгов с целью кредита

In [20]:
#Группировка данных по цели кредита
data_pivot_purpose = data.pivot_table(index=['purpose_category'], columns= 'debt', values='days_employed', aggfunc='count')
#Расчет доли невозврата
data_pivot_purpose['purpose_ratio_debt'] = data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1]) * 100

print('Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по цели кредита')
display(data_pivot_purpose['purpose_ratio_debt'].sort_values(ascending=False))

Процент заёмщиков, имеющих просрочку по кредиту, от общего числа клиентов. Группировка по цели кредита


purpose_category
операции с автомобилем      9.359034
получение образования       9.220035
проведение свадьбы          8.003442
операции с недвижимостью    7.233373
Name: purpose_ratio_debt, dtype: float64

Цели кредита заметно влияют на риски просрочки. Наиболее рискованные займы люди берут на покупку автомобиля и получение образования. Вероятность просрочки по обязательствам более, чем на 1% выше, чем при кредите на проведение свадьбы и на 2% выше, чем на операции с недвижимостью. 

<a id="conclusion2"></a>

### Вывод

В ходе исследовательского анализа данных были сделаны следующие выводы:
1. Количество детей у клиента влияет на вероятность просрочки по кредиту. Клиенты с детьми в среднем на 2% чаще имеют просрочку, чем бездетные
2. Семейное положение влияет на вероятность просрочки по кредиту. Клиенты, имевшие опыт юридически оформленного брака в среднем на 2% чаще имеют просрочку по кредитам, чем не имеющие такого опыта
3. Уровень дохода влияет на вероятность просрочки по кредиту. Клиенты, с более низким уровнем дохода в среднем на 1,5% чаще имеют просрочку, чем более обеспеченные клиенты
4. Цели кредита влияют на вероятность просрочки. Клиенты, покупающие автомобиль и получающие образование в среднем на 1-2% чаще имеют просрочку, чем клиенты, берущие кредит на свадьбу или операции с недвижимостью

<a id="conclusion_all"></a>

## Выводы и рекомендации

**Данные**

База данных содержит 21525 записей о клиентах банка в 12 колонках. Данные имеют пропуски (по 10% в полях days_employed и total_income), 54 явных и 71 не явный дубликат, аномалии (в полях days_employed и children), дублирующиеся столбцы (education и education_id, а так же family_status и family_status_id). 

На этапе предобработки данных, пропущенные значения были заменены на медианные значения, сгруппированные по типу занятости, аномальные значения заменены так же на средние или на наиболее близкие по смыслу. Тип данных в столбцах days_employed и total_income был заменен на целочисленный, дубликаты удалены (0,6% от размера базы данных), столбцы с дублирующейся информацией  education и family_status удалены. 

Данные о доходе клиентов и цели взятия кредита категоризированны в 5 групп.

**Анализ данных**

С целью выявления связи ряда параметров с вероятностью возврата кредита, было изучено несколько основных показателей: 
- количество детей, 
- семейное положение, 
- уровень дохода,
- цель кредита.

Диапазон значений по каждому оцениваемому параметру варьирует в пределах 2-3%, что говорит об отсутствии ярко выраженных контрастов в надежности тех или иных групп граждан.
Тем не менее, все изученные параметры показали связь с целевой переменной - наличием просрочки по кредиту. 

**Рекомендации бизнесу**

Наиболее рискованная группа клиентов, которые имеют немного повышенный риск просрочки по кредиту, имеет следующие параметры:
- Наличие детей
- Наличие брака в настоящем или прошлом
- Низкий доход
- Цель кредита - автомобить или образование

Наименее рискованная группа клиентов имеет следующие параметры:
- Нет детей
- Не состоят и не состояли в браке
- Доход выше среднего
- Цель кредита - свдаьба или недвижимость